## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
import re
import numpy as np

## Inputs

In [2]:
session_cookie = quote("UyfbwAuNgiGBW0Kls24zFdDxjkJhCZoFxw5Vy76OKto/YwnvlKRF1ZaqmqE9foWEEx1SamHVsDzGAc0aGM6uGhmdRyny6esb962ttVGDF89T1OsQwHq4NVwuasvgg70NowAMhqKnvek86bhM4J5o/Oe7xXR/VHKn1rIs62jBCmS/0YZwVZWyoBMrDYbw/IP6lk6Y+hU8Fx6qAk5Mx+9p0D4AdIPhypmERFU56ii6DZoUcL9ixUfbo8gNpbDF+JU0n/3o/46YfTDlPJDVy3HkuzA/Q9GwzkiRYvy0a2LMlP7buFEdIwJcom+S38CH3axZUJv0pMH4DyLVTLv5Y9idMcBJH1gq69xpIN+sqQ==--Pa3YFPf36fSY1YAh--tIcJOkufJL4qrIwDtEiVeQ==", safe = "")
#session_cookie = quote(input("Copy LW x AOC session cookie here: "), safe = "")
lines = open("input.txt").readlines()

### Extract Input

In [3]:
def extract_inputs(line):
    _, msg = line.strip().split(") ", 1)
    user_id, message = msg.split(": ", 1)
    return int(user_id), message

## AOC Interface

In [19]:
def get_user_rankings(user_id):
    '''
    Get daily rankings from day 1 through 25 for user_id.
    Returns two separate lists for parts 1 and 2.
    '''
    # Settings
    cookies = {
        "_lewagon_aoc_session": session_cookie,
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Accept": "text/html, application/xhtml+xml",
        "Accept-Language": "de,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://aoc.lewagon.community/",
        "Turbolinks-Referrer": "https://aoc.lewagon.community/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
    }
    url = "https://aoc.lewagon.community/stats/users/"
    # Initiate lists
    part_one = []
    part_two = []
    # Get data from server
    response = requests.get(f"{url}{user_id}", headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.findAll("table")[0]
    # Iterate over rows in table
    for idx, row in enumerate(table.findAll("tr")):
        if idx > 1:
            cells = row.findAll("td")
            part_one.append(cells[1].get_text())
            part_two.append(cells[4].get_text())
    # Reverse order and turn strings into numbers
    part_one = [int(c) if c.isnumeric() else -1 for c in part_one[::-1]]
    part_two = [int(c) if c.isnumeric() else -1 for c in part_two[::-1]]
    return part_one, part_two
print(get_user_rankings(134))

([12, 8, 12, 6, 2, 7, 5, 21, 4, 1, 27, 59, 20, 32, 11, 35, 11, -1, -1, -1, 36, 10, 16, -1, -1], [11, 6, 2, 3, 1, 2, 2, 8, 7, 5, 26, 55, 14, 22, 19, 31, 13, -1, -1, -1, -1, -1, -1, -1, -1])


## Encryption

### Product

In [5]:
def prod(key):
    '''
    Returns product of key elements
    '''
    a = 1
    for i in key:
        a *= i
    return a

### Get Cipher

In [6]:
def get_cipher(user_id, key):
    return int(str(abs(prod(key) + user_id * (1 if user_id % 2 == 0 else -1)))[::3])

### Get Ternary

In [7]:
def get_ternary(cipher):
    if cipher == 0:
        return "0"
    nums = []
    while cipher:
        cipher, r = divmod(cipher, 3)
        nums.append(str(r))
    return "".join(reversed(nums))

### Shuffle String

In [8]:
def shuffle_string(message, ter_cipher):
    for idx, val in enumerate(ter_cipher):
        if val == "0":
            message = message[::-1]
        elif val == "1":
            message = message[-1] + message[:-1]
        else:
            shift = (3**(len(ter_cipher) - idx - 1)) % len(message)
            message = message[shift:] + message[:shift]
    return message

### Prime Factor Sum

In [9]:
def get_prime_factor_sum(cipher):
    factors = []
    f = 2
    while f * f <= cipher:
        if cipher % f:
            f += 1
        else:
            cipher //= f
            factors.append(f)
    if cipher > 1:
        factors.append(cipher)
    return np.sum(factors) + (5 if len(factors) == 1 else 0)

## Jinglebet/Rudolphabet

### Get lyrics string

In [10]:
jinglebet_url = "https://www.41051.com/xmaslyrics/jingle.html"
rudolphabet_url = "https://www.41051.com/xmaslyrics/rudolph.html"
soup = BeautifulSoup(requests.get(rudolphabet_url).text, 'html.parser')
lyrics = re.sub("[^a-zA-Z]+", "", "".join(soup.find("span", {"class": "bodytext"}).get_text().split()))

### Extract characters in order

In [11]:
alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÇàéè0123456789 .!?,;:'-#/\()")
new_alphabet = ""
for char in lyrics:
    if char not in new_alphabet:
        new_alphabet += char
        alphabet.pop(alphabet.index(char))
new_alphabet += "".join(alphabet)

### Shift Alphabet

In [12]:
def shift_alphabet(shift):
    shift = shift % len(new_alphabet)
    shifted_alphabet = new_alphabet[-shift:] + new_alphabet[:-shift]
    return shifted_alphabet

### Shift Message

In [13]:
def shift_message(message, shifted_alphabet):
    return "".join([shifted_alphabet[new_alphabet.index(char)] for char in message])

## Run Encryption

In [14]:
f = open("output.txt", "w")
for line in lines:
    user_id, message = extract_inputs(line)
    _, dkey = get_user_rankings(user_id)
    cipher = get_cipher(user_id, dkey)
    ter_cipher = get_ternary(cipher)
    shifted_alphabet = shift_alphabet(get_prime_factor_sum(cipher))
    shifted_message = shift_message(message, shifted_alphabet)
    shuffled_message = shuffle_string(shifted_message, ter_cipher)
    f.write(f"{user_id}: {shuffled_message}\n")
f.close()